In [ ]:
import os.path as path
from datetime import datetime
import script.utility as util

In [ ]:
ar_log_file = "/raid/data/00_students/ug2196145/topcon_log_synchronizer.py/cam_pdr_student/ar/kairi1f1.csv"
inertial_log_file = "/raid/data/00_students/ug2196145/topcon_log_synchronizer.py/cam_pdr_student/inertial/kairi1f1_inertial_aggmgr.pkl"

ar_ts, ar_pos, ar_height, ar_ori = util.load_ar_log(ar_log_file)
inertial_ts, inertial_val = util.load_inertial_log(inertial_log_file)

find jump action

In [ ]:
min_jump_interval = 0

ar_jump_idxes = util.find_jump_in_topcon(ar_ts, ar_pos, ar_height, min_jump_interval)

In [ ]:
min_jump_interval = 0

inertial_jump_idxes = util.find_jump_in_inertial(inertial_ts, inertial_val[:, :3], min_jump_interval)

synchronize logs

In [ ]:
padding = 100
use_jump_idxes = "both"

cut_inertial_ts = util.cut_with_padding(inertial_ts, inertial_jump_idxes, padding)
cut_inertial_val = util.cut_with_padding(inertial_val, inertial_jump_idxes, padding)
resampled_ar_pos, resampled_ar_height, resampled_ar_ori = util.adjust_freq(cut_inertial_ts, util.adjust_ts_offset(inertial_jump_idxes, inertial_ts, ar_jump_idxes, ar_ts, use_jump_idxes), ar_pos, ar_height, ar_ori)

correct start point and direction

In [ ]:
ref_direct_tan = 0
rot_angle = 0

corrected_ar_pos = util.rot(rot_angle, resampled_ar_pos - resampled_ar_pos[0])
util.vis_tj(corrected_ar_pos, ref_direct_tan)

export synchronized log

In [ ]:
dir_and_file_name = path.splitext(path.basename(inertial_log_file))[0]

util.export_single(dir_and_file_name, cut_inertial_ts, cut_inertial_val, corrected_ar_pos, resampled_ar_height, resampled_ar_ori)
util.write_conf(dir_and_file_name, padding, rot_angle)